# Reading and writing Nastran output4 files

This and other notebooks are available here: ``kittyhawk:/home/loads/twidrick/code/pyyeti/docs/tutorials``.

First, do some imports:

In [ ]:
import numpy as np
import os
from pyyeti import op4

Generate a couple matrices to write to disk:

In [ ]:
a = np.random.randn(4, 10)
b = np.random.randn(5, 14)

---
#### Basic writing
Write them to a file in a random order. By default, the file with be a native-endian binary and the matrices will be written in dense (non-sparse) format.

In [ ]:
filename = 'rw_op4_demo.op4'
op4.write(filename, dict(a=a, b=b))

---
#### Basic reading
Read them back in and compare. The following reads the matrices into a dictionary:

In [ ]:
dct = op4.read(filename)
assert np.all(dct['a'] == a)
assert np.all(dct['b'] == b)

The "pretty-print" class from pyYeti can help in viewing the dictionary:

In [ ]:
from pyyeti.pp import PP
PP(dct);

---
If you need the form and type, set the `justmatrix` option to False (or use the `load` function ... it is the same as `read` except the default on `justmatrix`). In this case, dictionary entry is a 3-element tuple: (matrix, form, type):

In [ ]:
dct = op4.read(filename, justmatrix=False)
assert np.all(dct['a'][0] == a)
assert np.all(dct['b'][0] == b)
PP(dct);

---
#### Ascii save

As noted above, matrices are written in binary by default. To write in ASCII format, set `binary` to False:

In [ ]:
op4.write(filename, dict(a=a, b=b), binary=False)

In [ ]:
with open(filename) as f:
    for i in range(5):
        print(f.readline().strip())

---
#### Writing in a certain order
To ensure that "`b`" gets saved before "`a`" you have to use the list inputs.

In [ ]:
op4.write(filename, ['b', 'a'], [b, a], binary=False)
with open(filename) as f:
    for i in range(5):
        print(f.readline().strip())

---
#### Reading into list
If the output4 file has duplicate names, you'll have to read the variables into a list:

In [ ]:
b2 = b + 10
op4.write(filename, ['b', 'a', 'b'], [b, a, b2])
names, mats, *_ = op4.read(filename, into='list')
names

In [ ]:
assert np.all(b == mats[0])
assert np.all(a == mats[1])
assert np.all(b2 == mats[2])

---
#### Reading and writing sparse matrices
There are two aspects to "sparse" for this module:

1. There's the on-disk format. Matrices can be written in "dense" or one of the two sparse formats: either "bigmat" or "nonbigmat". The writing format is controlled by the `sparse` option of the ``write`` function.

2. There's also the in-memory format. Matrices can be read into regular ``numpy.ndarray`` matrices or into ``scipy.sparse`` sparse matrices. This is controlled by the `sparse` option of the ``read`` or ``load`` functions.

These two aspects are independent of each other. For example, ``numpy.ndarray`` matrices can be written in a sparse format and matrices written in the dense format can be read into ``scipy.sparse`` matrices.

To work with sparse matrices, we'll need the ``scipy.sparse`` module:

In [ ]:
import scipy.sparse as sp

First, create a 5 million by 5 million sparse matrix with just 3 elements to experiment with:

In [ ]:
data = [2.3, 5, -100.4]
rows = [2, 500000, 350000]
cols = [3750000, 500000, 4999999]
a = sp.csr_matrix((data, (rows, cols)), shape=(5000000, 5000000))
a

In [ ]:
print(a)

Save matrix to op4 file. Note: ``sparse='bigmat'`` is the default for ``scipy.sparse`` matrices. But, it doesn't hurt to specify it explicitly:

In [ ]:
op4.write(filename, dict(a=a), sparse='bigmat')

Read matrix back in. The default when reading *any* matrix is to create regular ``numpy.ndarray`` matrices. Therefore, the `sparse` option is required here:

In [ ]:
dct = op4.read(filename, sparse=True)
dct

All sparse matrices are returned in the COO format by default. To get the sparse matrix in the CSR format (for example) instead of the COO format, you can specify the `sparse` option as a two-tuple:

In [ ]:
dct = op4.read(filename, sparse=(True, sp.coo_matrix.tocsr))
dct

Clean up:

In [ ]:
os.remove(filename)